# CSTR operation or simulation data generation

In [2]:
import win32com.client as win32
aspen = win32.Dispatch('Apwn.Document.37.0')
aspen.InitFromFile(r"C:\Users\HP\Desktop\Project\Simulation\CSTR Simulation (SR).apw")
aspen.Engine.Run2()
aspen.Visible = 1

In [3]:
import win32com.client
import pandas as pd
import random

def run_simulation_and_save_data(num_simulations):
    data_list = []
    for _ in range(num_simulations):
        # Connect to Aspen Plus
        #aspen = win32com.client.Dispatch("Apwn.Document")
        
        # Generate random inputs
        e = random.uniform(50, 400)
        b = random.uniform(50, 400)
        b_temp = random.uniform(300, 500)
        b_pres = random.uniform(15, 50)
        e_temp = random.uniform(300, 500)
        e_pres = random.uniform(15, 50)
        r_temp = random.uniform(350, 450)
        r_pres = random.uniform(10, 50)
        r_vol = random.uniform(50, 300)
        
        # Set inputs in Aspen Plus
        aspen.Tree.FindNode("\Data\Streams\ETHYLENE\Input\FLOW\MIXED\ETHYLENE").SetValueUnitAndBasis(Value=e, unitcol=3, basis="MOLE-FLOW")
        aspen.Tree.FindNode("\Data\Streams\BENZENE\Input\FLOW\MIXED\BENZENE").SetValueUnitAndBasis(Value=b, unitcol=3, basis="MOLE-FLOW")
        aspen.Tree.FindNode("\Data\Streams\BENZENE\Input\TEMP\MIXED").SetValueUnitAndBasis(Value=b_temp, unitcol=1, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Streams\BENZENE\Input\PRES\MIXED").SetValueUnitAndBasis(Value=b_pres, unitcol=3, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Streams\ETHYLENE\Input\TEMP\MIXED").SetValueUnitAndBasis(Value=e_temp, unitcol=1, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Streams\ETHYLENE\Input\PRES\MIXED").SetValueUnitAndBasis(Value=e_pres, unitcol=3, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Blocks\B1\Input\TEMP").SetValueUnitAndBasis(Value=r_temp, unitcol=1, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Blocks\B1\Input\PRES").SetValueUnitAndBasis(Value=r_pres, unitcol=3, basis="No Attribute")
        aspen.Tree.FindNode("\Data\Blocks\B1\Input\VOL").SetValueUnitAndBasis(Value=r_vol, unitcol=3, basis="No Attribute")
        
        
        # Run simulation
        aspen.Engine.Run2()
        
        # Retrieve outputs from Aspen Plus
        res = aspen.Tree.FindNode("\Data\Blocks\B1\Output\TOT_RES_TIME").Value
        ethylene_out = aspen.Tree.FindNode("\Data\Streams\PRODUCTS\Output\MOLEFLOW\MIXED\ETHYLENE").Value
        benzene_out = aspen.Tree.FindNode("\Data\Streams\PRODUCTS\Output\MOLEFLOW\MIXED\BENZENE").Value
        ethylbenzene_out = aspen.Tree.FindNode("\Data\Streams\PRODUCTS\Output\MOLEFLOW\MIXED\ETHYLBEN").Value
        diethylbenzene_out = aspen.Tree.FindNode("\Data\Streams\PRODUCTS\Output\MOLEFLOW\MIXED\DIETHYLB").Value
        
        # Calculate additional outputs (conversion and yield)
        res_time = res*3600 #to convert the residence time from hours to seconds
        reacted = e - ethylene_out
        c = reacted / e
        con = c * 100
        y = ethylbenzene_out / e
        yd = y * 100 
        
        # Append data to list
        data_list.append({"ethylene_in": e, "benzene_in": b, "ethylene_temp": e_temp, "ethylene_pres": e_pres,
                          "benzene_temp": b_temp, "benzene_pres": b_pres, "reactor_temp": r_temp, "reactor_pres": r_pres, 
                          "reactor_volume" : r_vol, "ethylene_out": ethylene_out, "benzene_out": benzene_out,
                          "ethylbenzene_out": ethylbenzene_out, "diethylbenzene_out" : diethylbenzene_out,
                          "res_time": res_time, "conversion": con, "yield": yd})
        
    # Create DataFrame from list of dictionaries
    data = pd.DataFrame(data_list)
    
    #saving the data as CSV file
    data.to_csv('validation_dat002.csv', index = False)
    
    return data

simulation_data = run_simulation_and_save_data(200)
print(simulation_data)
    
    # Save DataFrame to CSV file
    #for i in range(num_simulations):
        #data.iloc[i:i+1].to_csv(f'simulation_data_{i+1}.csv', index=False)
    
    #return data

#run the simulation any number of times
#simulation_data = run_simulation_and_save_data(15)
#print(simulation_data)

# Specify the directory where you want to save the CSV files
#save_directory = "C:/Users/HP/Desktop/Project/Data"

# Run the simulation and save data
#simulation_data = run_simulation_and_save_data(5, save_directory)

     ethylene_in  benzene_in  ethylene_temp  ethylene_pres  benzene_temp  \
0     122.687373  358.387848     418.028222      38.269016    332.934155   
1     256.096421  226.731332     348.792248      38.077602    483.413582   
2     211.901263  337.417338     342.282347      26.770556    460.583138   
3      65.968522  301.908973     469.952647      33.299403    436.716913   
4     225.112508  247.134858     311.124445      35.662898    327.729237   
..           ...         ...            ...            ...           ...   
195   344.327419  111.906151     485.742061      23.463427    434.984516   
196   236.799274  354.893554     405.219135      39.090132    452.718282   
197   370.596947  189.209026     311.255141      17.114471    443.270448   
198   214.013618  270.492530     394.419337      40.563785    349.420853   
199   238.975282  316.836589     335.143052      49.161571    311.423850   

     benzene_pres  reactor_temp  reactor_pres  reactor_volume  ethylene_out  \
0       

In [99]:
# List of CSV file names
csv_files = ['combined_files.csv', 'simulation_data17.csv',]

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append to dfs list
for file in csv_files:
    dfs.append(pd.read_csv(file))

# Concatenate DataFrames along rows (axis=0)
concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)

# Write concatenated DataFrame to a new CSV file
concatenated_df.to_csv('combined_files.csv', index=False)

In [2]:
import pandas as pd
a = pd.read_csv('combined_files.csv')
a.tail(50)

,ethylene_in,benzene_in,ethylene_temp,ethylene_pres,benzene_temp,benzene_pres,reactor_temp,reactor_pres,reactor_volume,ethylene_out,benzene_out,ethylbenzene_out,diethylbenzene_out,res_time,conversion,yield
9950,295.334846,108.238247,347.842043,34.913516,470.910405,30.714958,434.029152,37.370517,203.074339,167.716852,2.303832,84.250837,21.683578,21.970172,43.211289,28.527225
9951,452.412283,376.270696,437.847676,32.480456,449.736388,17.108298,364.525991,12.105160,180.379988,329.983445,254.011241,122.090072,0.169383,9.900955,27.061343,26.986463
9952,92.396935,300.306975,385.541273,48.572893,335.559379,39.397677,423.792844,10.556876,291.713729,2.012351,210.120528,89.988308,0.198138,29.472303,97.822059,97.393175
9953,484.973549,468.382835,482.134469,48.836093,340.170217,31.274286,366.377315,41.681510,296.905432,302.913085,286.660812,181.383584,0.338440,14.338895,37.540287,37.400717
9954,264.489722,376.753467,393.526799,38.235636,372.179642,24.958745,376.300194,41.417850,243.495694,106.030867,218.668718,157.710642,0.374106,17.224010,59.911158,59.628269
9955,218.562443,79.266716,455.729459,26.937732,442.702717,28.866304,383.925477,25.908585,222.303415,147.210340,9.653956,67.873417,1.739343,35.191214,32.646095,31.054474
9956,63.470601,148.780988,303.162763,19.401374,318.801497,48.428370,376.277521,17.385515,250.155014,9.476510,94.887085,53.793715,0.100188,51.796129,85.069449,84.753750
9957,291.863572,144.572680,330.029799,38.937285,418.012052,42.751497,410.776102,46.589698,212.663731,148.572783,9.534766,126.785039,8.252875,23.168475,49.095126,43.439830
9958,473.144770,270.538927,305.986064,31.403806,403.578367,29.496379,424.468393,24.409505,182.474442,210.905261,22.015977,234.806391,13.716559,11.490588,55.424793,49.626754
9959,457.955439,186.957334,483.997319,42.235520,316.546355,30.655700,406.954588,48.424735,79.892087,310.515279,41.727415,143.019680,2.210240,5.369965,32.195307,31.230043


In [4]:
df0 = pd.read_csv("validation_data.csv")
df0.head(20)

,ethylene_in,benzene_in,ethylene_temp,ethylene_pres,benzene_temp,benzene_pres,reactor_temp,reactor_pres,reactor_volume,ethylene_out,benzene_out,ethylbenzene_out,diethylbenzene_out,res_time,conversion,yield
0,209.313091,123.974188,432.891707,49.450215,356.984311,44.795459,379.800912,44.544188,141.260061,120.412959,35.807134,87.433976,0.733078,20.457931,42.472323,41.771862
1,154.920410,375.142587,305.826810,47.758319,307.282543,16.060875,360.845351,49.714986,58.820787,123.126171,343.356476,31.777982,0.008129,4.575788,20.522951,20.512457
2,383.696997,393.917093,438.509811,38.132210,361.652885,43.623407,404.588385,16.525729,295.184387,90.664892,104.334801,286.132479,3.449813,18.140803,76.370706,74.572509
3,232.349326,327.634248,438.224028,21.866493,341.169813,48.166255,440.437868,24.751593,114.906643,16.841136,114.437527,210.885252,2.311469,9.094404,92.751803,90.762153
4,359.896282,136.160392,352.745431,19.318662,435.923506,47.553076,373.912590,19.937432,83.499870,294.461089,70.916626,65.052339,0.191427,7.342121,18.181681,18.075302
5,163.571422,126.196520,427.288107,35.819403,318.395964,33.728386,408.627611,43.820292,241.119912,43.285897,11.067476,109.972562,5.156482,41.376282,73.537005,67.232137
6,89.003311,80.343910,301.295621,37.846179,436.889264,49.972701,397.176122,37.823332,132.541921,21.883798,14.422926,64.722456,1.198528,38.681546,75.412378,72.719156
7,367.592314,107.579812,364.400416,28.254739,369.295258,28.858813,369.312163,49.130574,269.180684,289.588280,30.175021,76.805548,0.599243,25.598675,21.220257,20.894220
8,276.891706,360.781226,493.232291,35.559428,371.611590,29.853870,416.498404,15.849690,110.197596,55.727446,141.253666,217.890860,1.636700,7.817536,79.873920,78.691725
9,171.740637,342.682964,477.830911,19.781141,371.195102,39.641429,353.788720,27.332670,229.805871,107.408717,278.388763,64.256482,0.037719,19.377713,37.458764,37.414839


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("combined_files.csv")
df.head(7)

,ethylene_in,benzene_in,ethylene_temp,ethylene_pres,benzene_temp,benzene_pres,reactor_temp,reactor_pres,reactor_volume,ethylene_out,benzene_out,ethylbenzene_out,diethylbenzene_out,res_time,conversion,yield
0,146.720325,142.081939,343.119700,39.573407,382.385275,16.047851,406.950797,20.390476,258.489453,23.552579,21.790994,117.414144,2.876801,44.173775,83.947296,80.025821
1,79.761171,87.012310,390.286754,43.972875,304.382421,18.332004,377.473155,26.757742,224.147713,19.160227,26.855682,59.712312,0.444316,66.053032,75.978002,74.863886
2,130.291538,59.432419,419.465346,49.112356,437.669690,24.637221,401.395466,38.474881,230.776326,67.987771,2.375962,51.809146,5.247311,59.267673,47.818737,39.764014
3,178.961231,69.362396,335.582612,39.366606,413.362034,43.318462,412.617143,43.768866,124.379630,107.884259,3.542665,60.562488,5.257243,22.733416,39.716408,33.841121
4,148.649473,192.679442,306.812503,31.359526,491.509726,37.101147,400.088948,40.232040,261.692332,18.564178,63.673407,127.926775,1.079260,36.325547,87.511440,86.059353
5,112.399420,105.440769,496.175667,30.712742,341.906161,41.464772,404.830028,12.282140,134.053180,24.233077,18.958461,84.798274,1.684035,30.123909,78.440212,75.443693
6,66.670806,186.691641,430.256429,32.245598,426.234509,32.027542,380.515870,18.620202,277.921219,8.021562,128.134006,58.466026,0.091609,46.893736,87.968404,87.693594
